# Debug

Force no GPU. Used for local development. Remove when submitting notebook to Keras in PR.

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Introduction

This example shows how to train a neural network to recognize text in images.  We use Keras's functional API to implement a model based on the CRNN architecture. We also create both a custom loss function to adapt the CTC loss provided by `tensorflow.keras.backend`, as well as a custom metric to adapt `tf.edit_distance`. 

To demonstrate this setup, we build a training dataset by generating synthetic images from a source list of words. To implement the dataset, we subclass `tensorflow.keras.utils.Sequence`. It provides an `on_epoch_end` callback, which we use for curriculum learning by gradually ramping up the difficulty of the dataset during training.

## Setup

One external package is required for this example.
* `cairocffi` provides Python bindings for `cairo`, a 2D vector graphics library written in the C programming language. We use this package to generate synthetic images of text.

If you've opened this example in Google Colab, you can install the package like this:

In [ ]:
!pip install cairocffi


Then, import the required dependencies:

In [ ]:
# TensorFlow packages
from tensorflow.keras import layers, losses, models, optimizers, utils
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
import tensorflow as tf

# Third-party packages
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import cairocffi as cairo

## Preparing a synthetic dataset

We train our neural network on a dataset that contains synthetic images. The synthetic images are generated on the fly from a list of source words. 

> _**Note:** Other text-image datasets exist, some of which are widely used in OCR research. However, these datasets can be quite large, making them difficult to handle in a code example. If you plan to adapt this tutorial for your own work, consider using one of the following datasets instead: [MJSynth (MJ)](http://www.robots.ox.ac.uk/~vgg/data/text/), [SynthText (ST)](http://www.robots.ox.ac.uk/~vgg/data/scenetext/), [IIIT](http://cvit.iiit.ac.in/projects/SceneTextUnderstanding/IIIT5K.html), [SVT](http://www.iapr-tc11.org/mediawiki/index.php/The_Street_View_Text_Dataset), [IC03](http://www.iapr-tc11.org/mediawiki/index.php/ICDAR_2003_Robust_Reading_Competitions), [IC13](http://rrc.cvc.uab.es/?ch=2), [IC15](http://rrc.cvc.uab.es/?ch=4), [SVTP](http://openaccess.thecvf.com/content_iccv_2013/papers/Phan_Recognizing_Text_with_2013_ICCV_paper.pdf), and/or [CUTE](http://cs-chan.com/downloads_CUTE80_dataset.html)._



### Downloading the source word lists

First, let's download and uncompress an archive containing two text files.

In [ ]:
from pathlib import Path

tgz_path = Path(
    utils.get_file('wordlists.tgz',
                   origin='http://www.mythic-ai.com/datasets/wordlists.tgz',
                   untar=True)
)
dataset_dir = tgz_path.parent

wordlists = [f.name for f in dataset_dir.iterdir() if f.suffix == '.txt']
print(f".txt files contained in wordlists.tgz: {wordlists}")

monogram_file = dataset_dir / "wordlist_mono_clean.txt"
bigram_file   = dataset_dir / "wordlist_bi_clean.txt"


Each file contains a different set of [**n-gram**](https://en.wikipedia.org/wiki/N-gram) data samples:
* `wordlist_mono_clean.txt` contains a list of **monograms**, or single words. These words have been pre-sorted according to their frequency in English speech.
* `wordlist_bi_clean.txt` contains a list of **bigrams**, or pairs of words.

In [ ]:
monogram_sample, bigram_sample = [], []
i = 5
with monogram_file.open() as f_m, bigram_file.open() as f_b: 
    for _ in range(i):
        monogram_sample.append(f_m.readline().rstrip())
        bigram_sample.append(f_b.readline().rstrip())

print(f"First {i} monograms: {monogram_sample}")
print(f"First {i} bigrams: {bigram_sample}")


### Loading and preprocessing words

Before using these word lists to create a dataset, we filter out any unsuitable examples as we load them into memory. 

In [ ]:
# Criteria 1: Text sequences must be under a specified length
ABSOLUTE_MAX_STRING_LEN = 16

# Criteria 2: Text sequences must contain only specified characters
ALPHABET = u'abcdefghijklmnopqrstuvwxyz '

import re

def preprocess_words(wordlist_file, valid_characters, max_string_len=None):
    def _is_valid_str(in_str):
        search = re.compile(rf'^[{valid_characters} ]+$', re.UNICODE).search
        return bool(search(in_str))

    def _is_length_of_word_valid(word):
        return (max_string_len == -1 or
                max_string_len is None or
                len(word) <= max_string_len)

    valid_words = []
    with wordlist_file.open() as f:
        for line in f:
            word = line.rstrip()
            if _is_valid_str(word) and _is_length_of_word_valid(word):
                valid_words.append(word)

    return valid_words

monograms = preprocess_words(monogram_file, ALPHABET, ABSOLUTE_MAX_STRING_LEN)
bigrams   = preprocess_words(bigram_file, ALPHABET, ABSOLUTE_MAX_STRING_LEN)

print(f"Wordlists contain {len(monograms)} valid monograms" 
      f" and {len(bigrams)} valid bigrams.")


### Set aside words for validation

Because the monograms have been ordered by frequency, simply slicing the dataset using Python's list indexing will cause easy and difficult words to be distributed unevenly. We'll instead define a function that uses a slightly more involved approach.


In [ ]:
VALID_SPLIT = 0.2

def valid_split(input_list, split_ratio):
    n = round(1 / split_ratio)
    trn_list, val_list = [], []
    for i, item in enumerate(input_list):
        if i % n == 0:
            val_list.append(item)
        else:
            trn_list.append(item)

    return trn_list, val_list

trn_monograms, val_monograms = valid_split(monograms, VALID_SPLIT)
trn_bigrams, val_bigrams = valid_split(bigrams, VALID_SPLIT)


### Defining functions for creating synthetic text images

These functions generate images from strings of text. They include image transformations to introduce variations in the generated images, too. This helps to improve the capacity of the model, and lower the risk of overfitting.

In [ ]:
# this creates larger "blotches" of noise which look
# more realistic than just adding gaussian noise
# assumes greyscale with pixels ranging from 0 to 1

def speckle(img):
    severity = np.random.uniform(0, 0.6)
    blur = ndimage.gaussian_filter(np.random.randn(*img.shape) * severity, 1)
    img_speck = (img + blur)
    img_speck[img_speck > 1] = 1
    img_speck[img_speck <= 0] = 0
    return img_speck


# paints the string in a random location the bounding box
# also uses a random font, a slight random rotation,
# and a random amount of speckle noise

def paint_text(text, w, h, rotate=False, ud=False, multi_fonts=False):
    surface = cairo.ImageSurface(cairo.FORMAT_RGB24, w, h)
    with cairo.Context(surface) as context:
        context.set_source_rgb(1, 1, 1)  # White
        context.paint()
        # this font list works in CentOS 7
        if multi_fonts:
            fonts = [
                'Century Schoolbook', 'Courier', 'STIX',
                'URW Chancery L', 'FreeMono']
            context.select_font_face(
                np.random.choice(fonts),
                cairo.FONT_SLANT_NORMAL,
                np.random.choice([cairo.FONT_WEIGHT_BOLD, cairo.FONT_WEIGHT_NORMAL]))
        else:
            context.select_font_face('Courier',
                                     cairo.FONT_SLANT_NORMAL,
                                     cairo.FONT_WEIGHT_BOLD)
        context.set_font_size(25)
        box = context.text_extents(text)
        border_w_h = (4, 4)
        if box[2] > (w - 2 * border_w_h[1]) or box[3] > (h - 2 * border_w_h[0]):
            print(text)
            print(box[0])
            print(box[1])
            raise IOError(('Could not fit string into image.'
                           'Max char count is too large for given image width.'))

        # teach the RNN translational invariance by
        # fitting text box randomly on canvas, with some room to rotate
        max_shift_x = w - box[2] - border_w_h[0]
        max_shift_y = h - box[3] - border_w_h[1]
        top_left_x = np.random.randint(0, int(max_shift_x))
        if ud:
            top_left_y = np.random.randint(0, int(max_shift_y))
        else:
            top_left_y = h // 2
        context.move_to(top_left_x - int(box[0]), top_left_y - int(box[1]))
        context.set_source_rgb(0, 0, 0)
        context.show_text(text)

    buf = surface.get_data()
    a = np.frombuffer(buf, np.uint8)
    a.shape = (h, w, 4)
    a = a[:, :, 0]  # grab single channel
    a = a.astype(np.float32) / 255
    a = np.expand_dims(a, 0)
    if rotate:
        a = image.random_rotation(a, 3 * (w - top_left_x) / w + 1)
    a = speckle(a)

    return a

### Building the dataset using the `Sequence` class

In [ ]:
BATCH_SIZE = 32
IMG_H = 64
IMG_W = 128
DOWNSAMPLE_FACTOR = 4

SUBSET_NUM = 12800     # Number of words to pull from source lists into a subset
MAX_WORD_LEN = 4       # Starting max word length within subset
BLANK_RATIO = 0.2      # Ratio of blanks to words
MONO_BI_RATIO = 1      # Ratio of monograms to bigrams

In [ ]:
import math
import random

class TextImageSequence(utils.Sequence):
    def __init__(self, monograms, bigrams, batch_size, 
                 img_w, img_h, downsample_factor,
                 subset_num=-1, max_word_len=4, mono_bi_ratio=1, 
                 blank_ratio=0, start_epoch=0):
        self.batch_size = batch_size
        self.epoch_num = start_epoch
        self.downsample_factor = downsample_factor

        # Config settings for synthetic image generation function
        self.img_w = img_w 
        self.img_h = img_h 
        self.rotate=False
        self.ud=False
        self.multi_fonts=False
        
        # Base wordlists
        self.monograms = monograms
        self.bigrams = bigrams

        # Config settings for creating a wordlist subset
        self.subset_num = subset_num
        self.max_word_len = max_word_len
        self.mono_bi_ratio = mono_bi_ratio
        self.blank_ratio = blank_ratio

        self._build_word_subset()

    def __len__(self):
        return math.ceil(len(self.X) // self.batch_size)
        
    def __getitem__(self, index):
        # Width and height are backwards from typical Keras convention,
        # because width is the time dimension when it gets fed into the RNN
        X_batch = np.ones([self.batch_size, self.img_w, self.img_h, 1])
        y_batch = np.ones([self.batch_size, ABSOLUTE_MAX_STRING_LEN + 2])
        
        # TODO: For loop -> list comprehension / fancy slicing / map
        for i in range(self.batch_size):
            X_batch[i, 0:self.img_w, :, 0] = (
                paint_text(self.X[index + i],
                           self.img_w, self.img_h,
                           self.rotate,
                           self.ud,
                           self.multi_fonts)[0, :, :].T
            )
            y_batch[i] = self.y[index + i]
        
        return X_batch, y_batch

    def _text_to_labels(self, text):
        # to use k.backend.ctc_batch_loss, 3 pieces of information are needed:
        #   1. y_true (text, in integer form, padded out to max string length)
        #   2. sequence length for each batch item in y_true
        #   3. sequence length for each batch item in y_pred
        if text == ' ':
            y_true = ([len(ALPHABET)] +                            # blank val
                      [-1]*(ABSOLUTE_MAX_STRING_LEN - len(text)))  # padding
        else:
            y_true = ([ALPHABET.find(c) for c in text] +           # char -> int
                      [-1]*(ABSOLUTE_MAX_STRING_LEN - len(text)))  # padding
        y_true_len = [len(text)]
        y_pred_len = [self.img_w // self.downsample_factor - 2]

        # Keras losses only accept 2 inputs (label, y_pred), so we
        # concatenate the 3 pieces of information into a single label
        label = y_pred_len + y_true_len + y_true

        return label

    def _build_word_subset(self):
        string_list=  []

        # Enforce ratio of blanks:words, and ratio of monograms:bigrams
        num_blanks = round(self.subset_num * self.blank_ratio)
        num_words = self.subset_num - num_blanks
        num_monograms = round(num_words * self.mono_bi_ratio)

        for monogram in self.monograms:
            if len(string_list) == num_monograms and self.subset_num is not -1:
                break
            if len(monogram) <= self.max_word_len:
                string_list.append(monogram)

        for bigram in self.bigrams:
            if len(string_list) == num_words and self.subset_num is not -1:
                break
            if len(bigram) <= self.max_word_len:
                string_list.append(bigram)

        for _ in range(num_blanks):
            string_list.append(' ')
        
        if len(string_list) < self.subset_num:
            # Possibly raise only Warning? Not a fatal issue...
            raise IOError(f'Could only pull {len(string_list)} words into '
                            f'subset when {self.subset_num} were requested.')
            
        random.shuffle(string_list)

        self.X = string_list
        self.y = [self._text_to_labels(t) for t in string_list]
        
    def on_epoch_end(self):
        # Update paint function parameters to implement curriculum learning
        self.epoch_num += 1
        if self.epoch_num >= 3:
            self.ud=True
        if self.epoch_num >= 6:
            self.multi_fonts=True
        if self.epoch_num >= 9:
            self.rotate=True

        # Hypothetical implementation for other curriculum learning setups
        # if self.epoch_num % 2 == 0:
        #     # Ramp up the length of allowable words to max
        #     if self.max_word_len < ABSOLUTE_MAX_STRING_LEN:
        #         self.max_word_len += 1

        #     # Ramp up towards a 50/50 split of monograms and bigrams
        #     self.mono_bi_ratio -= (self.mono_bi_ratio - 0.5) * 0.1

        #     # Rebuild the wordlist
        #     self._build_word_subset()

Create Sequence dataset

In [ ]:
trn_dataset = TextImageSequence(trn_monograms, trn_bigrams, BATCH_SIZE, 
                                IMG_W, IMG_H, DOWNSAMPLE_FACTOR, SUBSET_NUM,
                                MAX_WORD_LEN, MONO_BI_RATIO, BLANK_RATIO)
val_dataset = TextImageSequence(val_monograms, val_bigrams, BATCH_SIZE, 
                                IMG_W, IMG_H, DOWNSAMPLE_FACTOR)


Visualize images from a sample batch

In [ ]:
sample_batch = next(iter(trn_dataset))

f, axarr = plt.subplots(5, 2)
for i, ax in enumerate(f.axes):
    # Image is in (W, H, 1) format. Squeeze changes this to (W, H), and .T
    # transposes it to (H, W), allowing it to be displayed as grayscale image
    ax.imshow(np.squeeze(sample_batch[0][i]).T, cmap='gray', vmin=0, vmax=1)


## Preparing the model, optimizer, loss function, and metrics

### Model setup

In [ ]:
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 512
act = 'relu'

# Input layers ()
X = layers.Input(shape=(IMG_W, IMG_H, 1), dtype='float32')

# Convolution layers
inner = layers.Conv2D(conv_filters, kernel_size, padding='same',
                activation=act, kernel_initializer='he_normal',
                name='conv1')(X)
inner = layers.MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
inner = layers.Conv2D(conv_filters, kernel_size, padding='same',
                activation=act, kernel_initializer='he_normal',
                name='conv2')(inner)
inner = layers.MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

conv_to_rnn_dims = (IMG_W // (pool_size ** 2),
                    (IMG_H // (pool_size ** 2)) * conv_filters)
inner = layers.Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

# cuts down input size going into RNN:
inner = layers.Dense(time_dense_size, activation=act, name='dense1')(inner)

# Recurrent layers: Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = layers.GRU(rnn_size, return_sequences=True,
            kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = layers.GRU(rnn_size, return_sequences=True,
                go_backwards=True, kernel_initializer='he_normal',
                name='gru1_b')(inner)
gru1_merged = layers.add([gru_1, gru_1b])
gru_2 = layers.GRU(rnn_size, return_sequences=True,
            kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = layers.GRU(rnn_size, return_sequences=True, go_backwards=True,
                kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

# transforms RNN output to character activations:
inner = layers.Dense(len(ALPHABET)+1, kernel_initializer='he_normal',
                name='dense2')(layers.concatenate([gru_2, gru_2b]))
y_pred = layers.Activation('softmax', name='softmax')(inner)

model = models.Model(inputs=X, outputs=y_pred)
model.summary()


### Optimizer

In [ ]:
sgd = optimizers.SGD(learning_rate=0.001,
                     decay=1e-6,
                     momentum=0.9,
                     nesterov=True,
                     clipnorm=5)


### Loss function

In [ ]:
def ctc_loss(y, y_pred):
    # Decompose label into its subsequent parts
    input_length = y[:, 0:1]
    label_length = y[:, 1:2]
    y_true = y[:, 2:] 

    # From old example: "the first couple outputs of the RNN tend to be garbage"
    y_pred = y_pred[:, 2:, :]
    
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)


### Evaluation metrics

Adapt old functionality to custom Keras metric:

In [ ]:
from IPython.core.debugger import set_trace

def edit_distance(y, y_pred):
    # Decompose label into its subsequent parts
    input_length = y[:, 0:1]
    label_length = y[:, 1:2]
    y_true = y[:, 2:]

    # This form is required by ctc_label_dense_to_sparse and ctc_greedy_decoder
    input_length = tf.cast(tf.squeeze(input_length, axis=-1), tf.int32)
    label_length = tf.cast(tf.squeeze(label_length, axis=-1), tf.int32)

    # Get int64 SparseTensor representation of y_true
    y_true_sparse = K.ctc_label_dense_to_sparse(y_true, label_length)
    y_true_sparse = tf.cast(y_true_sparse, tf.int64)

    # Get int64 SparseTensor representation of y_pred
    y_pred = tf.transpose(y_pred, [1, 0, 2]) 
    decoded, neg_sum_logits = tf.nn.ctc_greedy_decoder(y_pred, input_length)
    y_pred_sparse = decoded[0]

    # Calculate Levenshtein distance for batch
    dist_batch = tf.edit_distance(y_pred_sparse, y_true_sparse)

    return tf.reduce_mean(dist_batch, axis=-1)


## Bringing it all together: the training routine

In [ ]:
model.compile(loss=ctc_loss, optimizer=sgd, metrics=[edit_distance])
model.fit(trn_dataset, batch_size=BATCH_SIZE, epochs=20, shuffle=True,
          validation_data=val_dataset)
